In [ ]:
# Pytorch
import torch
#nn: 신경망을 구축하는데 필요한 모듈, 
import torch.nn as nn
# functional: 활성화 함수나 풀링 등 레이어들과 유사한 기능을 수행하는 함수 제공
import torch.nn.functional as F
# transformers: 허깅페이스(Hugging Face)에서 제공하는 라이브러리로, 
    # 최신 트랜스포머 기반 모델들을 매우 쉽게 사용할 수 있도록 돕는 강력한 도구
# T5ForConditionalGeneration: 번역, 요약과 같은 '조건부 생성' 작업을 위해 특별히 설계된 클래스
# T5Tokenizer: T5 모델을 위한 전용 토크나이저입니다.
# Trainer: 델, 훈련 설정, 데이터셋 등을 입력받아 전체 훈련 및 평가 과정을 자동으로 처리해주는 고수준 API입니다.
# TrainingArguments : 모델 훈련에 필요한 모든 하이퍼파라미터(학습률, 배치 크기, 에폭 수, 저장 경로 등)를 정의하는 설정 클래스입니다
# PreTrainedModel: 허깅페이스의 모든 사전 훈련된 모델이 상속하는 기본 클래스입니다.
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, PreTrainedModel
# 퀀스-투-시퀀스(Seq2Seq) 모델의 출력값을 체계적으로 정리해주는 데이터 구조
from transformers.modeling_outputs import Seq2SeqLMOutput
# Dataset: 사용자 정의 데이터셋을 만들기 위한 추상 클래스입니다.
# DataLoader: 지정된 배치 크기(batch size)에 맞게 데이터를 묶어주고, 데이터를 섞거나(shuffle), 여러 개의 CPU 코어를 사용해 병렬로 데이터를 불러오는 등 훈련 과정을 효율적으로 만들어주는 핵심적인 유틸리티입니다.
from torch.utils.data import Dataset, DataLoader
# 구글에서 개발한 비지도(unsupervised) 텍스트 토크나이저 라이브러리입니다. BPE, Unigram 등 다양한 서브워드(subword) 분절 알고리즘을 지원하며, 특정 언어에 종속되지 않는다는 장점이 있습니다.
import sentencepiece as spm
# 운영체제(Operating System)와 상호작용할 수 있는 다양한 기능을 제공하는 파이썬 표준 라이브러리입니다.
import os

# 재현성을 위한 시드 고정
torch.manual_seed(42)

c:\Users\Hyuk\anaconda3\envs\KSEB\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# --- 2.1. 이력서 텍스트 파일 준비 (예시) ---
# 'resumes.txt' 파일에 이력서 관련 문장들을 한 줄에 하나씩 저장합니다.
# 예: "삼성전자에서 반도체 설계 엔지니어로 3년간 근무했습니다."
#     "파이썬과 PyTorch를 활용한 딥러닝 모델 개발 경험이 있습니다."

# --- 2.2. SentencePiece 토크나이저 훈련 ---
def train_custom_tokenizer(data_file, model_prefix='resume_tokenizer', vocab_size=32000):
    '''
        data_file: Required. Path to the text file to train the torquerizer on.
            필수 입력값. 토크나이저를 훈련시킬 텍스트 파일의 경로
        model_prefix: Optional input. The name of the model file that will be generated after training. The default value is set to ‘resume_tokenizer’
            선택 입력값. 훈련 후 생성될 모델 파일의 이름입니다. 기본값은 'resume_tokenizer'로 설정되어 있습니다.

        vocab_size: Optional input. Determines the size of the word field. The default is 32000.
            선택 입력값. 단어장의 크기를 결정합니다. 기본값은 32000개 입니다.
    '''
    if not os.path.exists(f'{model_prefix}.model'):
        print("맞춤형 토크나이저를 훈련합니다...")
        spm.SentencePieceTrainer.train(
            f'--input={data_file} --model_prefix={model_prefix} '
            f'--vocab_size={vocab_size} --model_type=bpe '
            f'--character_coverage=0.9995 '
            f'--pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3'
        )
        print("토크나이저 훈련 완료!")
    else:
        print("훈련된 토크나이저를 사용합니다.")

# 이력서 데이터 파일 경로
resume_data_file = 'resumes.txt' 
# # 이력서 데이터가 없다면, 임시 파일 생성
if not os.path.exists(resume_data_file):
    with open(resume_data_file, 'w', encoding='utf-8') as f:
        f.write("삼성전자에서 반도체 설계 엔지니어로 3년간 근무했습니다.\n")
        f.write("파이썬과 PyTorch를 활용한 딥러닝 모델 개발 경험이 있습니다.\n")
        f.write("국민대학교 컴퓨터공학부를 졸업했습니다.\n")


train_custom_tokenizer(resume_data_file)

# 훈련된 토크나이저 로드
tokenizer = T5Tokenizer(vocab_file='resume_tokenizer.model', extra_ids=0)

# --- 2.3. 지문자 처리를 위한 자모(Jamo) 토큰 추가 ---
# 한글 자모와 같은 특수 토큰들을 사전에 추가합니다.
korean_jamo = [
    'ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ',
    'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ'
]

# korean_jamo = [
#     'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ',
#     'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ'
# ]
tokenizer.add_tokens(korean_jamo)
print(f"새로운 토큰 추가 후 토크나이저 크기: {len(tokenizer)}")